# Music Genre Classifier


In [ ]:
%pip install wget

In [ ]:
import os
import cv2
import wget
import gdown
import random
import shutil
import tarfile
import numpy as np
import pandas as pd
import tensorflow as tf
from pandas import DataFrame
from keras import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Activation

In [ ]:
SEED_VALUE = 42

# Fix seed to make training deterministic.
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

In [ ]:
if os.getenv('COLAB_RELEASE_TAG'):
    from google.colab import drive 
    drive.mount('/content/gdrive')

In [ ]:
FEATURES = ['mfcc', 'mfcc_bands', 'mfcc_bands_log']
GENRES = ['blues', 'classical', 'country', 'disco', \
    'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

## Load the GTZAN Dataset


In [ ]:
# download the features from Google Drive
if os.getenv('COLAB_RELEASE_TAG'):
    gdown.download(
        url='https://drive.google.com/file/d/1LDlvwUeJ-h3JntNSmQABNV8z8oqltojO/view?usp=drive_link', 
        output='gtzan-features.zip', 
        quiet=False, fuzzy=True)
elif 'gtzan-features.zip' not in os.listdir('.'):
    raise Exception("Download the GTZAN features.")

# descompress file
shutil.unpack_archive(filename='gtzan-features.zip', extract_dir="gtzan")


## Dataset


In [ ]:
def load_dataset(src: str, set: str,  feature: str):
    """
        Divide the dataset into two subsets, the training set, \
        and the validation set.
    """
    dataset = []
    
    for genre in GENRES:
        for img in os.listdir(f'{src}/{set}/{feature}/{genre}'):
            img_cv = cv2.imread(f'{src}/{set}/{feature}/{genre}/{img}')
            img_cv = cv2.resize(img_cv, (256, 192))
            img_cv = np.array(img_cv, dtype=np.float32)
            dataset.append([img_cv, genre])

    df = DataFrame(data=np.array(dataset, dtype=object), columns=[feature, 'genre'])

    one_hot = pd.get_dummies(df['genre'])

    df = pd.concat([df, one_hot], axis=1)

    df.drop(['genre'], axis=1, inplace=True)
    
    return (np.array([tf.convert_to_tensor(img) for img in df[feature]]), df[GENRES])

In [ ]:
X_train, y_train = load_dataset('./gtzan/', 'training', 'mfcc')
X_val, y_val = load_dataset('./gtzan/', 'validation', 'mfcc')

## CNN


In [ ]:
model = Sequential()

model.add(Conv2D(input_shape=(192, 256, 3), filters=16, kernel_size=(3,3), activation='relu', padding='same', strides=1))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = []

model_storage = '/content'

if os.getenv('COLAB_RELEASE_TAG'):
    model_storage = '/content/gdrive/MyDrive'

if 'model_storage' not in os.listdir(model_storage):
    os.mkdir(f'{model_storage}/model_storage')

last_iter = 0

if len(os.listdir(f'{model_storage}/model_storage')) > 0:
    last_iter = sorted([int(model.removeprefix('model_').removesuffix('.keras'))
        for model in os.listdir(f'{model_storage}/model_storage')], reverse=True)[0]
    model = tf.keras.saving.load_model(f'{model_storage}/model_storage/model_{last_iter}.keras')

for i in range(last_iter, 50):
    history.append(model.fit(x=X_train, y=y_train, epochs=10, validation_data=(X_val, y_val)))
    model.save(f'{model_storage}/model_storage/model_{i}.keras')